<a href="https://colab.research.google.com/github/OctopusCosmic/Merck-Computer-Vision/blob/main/Yolov5_DeepSort_Multiple_Bag_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Clone repo, install dependencies and check PyTorch and GPU.

In [2]:
!git clone --recurse-submodules https://github.com/mikel-brostrom/Yolov5_DeepSort_Pytorch.git  # clone repo
%cd Yolov5_DeepSort_Pytorch
%pip install -qr requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.10.0+cu111 (CPU)


# Crop data

Get test video from repo and extract the first 2 seconds of it 

In [12]:
# get yolov5m model trained on the crowd-human dataset
#!wget -nc https://github.com/mikel-brostrom/Yolov5_DeepSort_Pytorch/releases/download/v.2.0/crowdhuman_yolov5m.pt -O /content/Yolov5_DeepSort_Pytorch/yolov5/weights/crowdhuman_yolov5m.pt
#!wget -nc /content/best\ (3).pt -O /content/Yolov5_DeepSort_Pytorch/yolov5/weights/bag_yolov5m.pt

# get the test video from the repo
#!wget -nc https://github.com/mikel-brostrom/Yolov5_DeepSort_Pytorch/releases/download/v.2.0/test.avi
# extract 3 seconds worth of video frames of it

#!y | ffmpeg -ss 00:00:00 -i test.avi -t 00:00:02 -c copy out.avi
!y | ffmpeg -ss 00:00:00 -i /content/Multiple_Bags_film1.mp4 -t 00:00:02 -c copy out.mp4

/bin/bash: y: command not found
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxm

## Run inference on video

The ``cv2.imshow()`` and ``cv.imshow()`` functions from the [opencv-python](https://github.com/skvark/opencv-python) package are incompatible with Jupyter notebook; see https://github.com/jupyter/notebook/issues/3935. 

Hence we chose to save it to file in this notebook. Locally you can use the ``--show-vid`` flag in order visualize the tracking in real-time

In [13]:
#!python track.py --yolo_model /content/Yolov5_DeepSort_Pytorch/yolov5/weights/crowdhuman_yolov5m.pt --source out.avi --save-vid
!python track.py --yolo_model /content/bag_model.pt --source out.mp4 --save-vid

deep_sort/deep/reid/torchreid/metrics/rank.py:12: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  'Cython evaluation (very fast so highly recommended) is '
YOLOv5 🚀 v6.0-159-gdb6ec66 torch 1.10.0+cu111 CPU

Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_x0_25_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
Model: osnet_x0_25
- params: 203,568
- flops: 82,316,000
YOLOv5 🚀 v6.0-159-gdb6ec66 torch 1.10.0+cu111 CPU

Fusing layers... 
Model Summary: 213 layers, 7015519 parameters, 0 gradients
video 1/1 (1/62) /content/Yolov5_DeepSort_Pytorch/out.mp4: 384x640 1 Flour bag, 1 Sugar bag, Done. YOLO:(0.311s), DeepSort:(0.046s)
video 1/1 (2/62) /content/Yolov5_DeepSort_Pytorch/out.mp4: 384x640 1 Flour bag, 1 Sugar bag, Done. YOLO:(0.298s), DeepSort:(0.059s)
video 1/1 (3/62) /content/Yolov5_DeepSort_Pytorch/out.m

# Show results

https://stackoverflow.com/questions/60977179/how-to-play-avi-file-in-google-colab

Convert avi to mp4

In [11]:
#!ffmpeg -i /content/Yolov5_DeepSort_Pytorch/runs/track/exp/out.avi output.mp4

Get the file content into data_url

In [9]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/Yolov5_DeepSort_Pytorch/runs/track/exp/out.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

Display it with HTML

In [ ]:
HTML("""
<video controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)